# Construção da base final - Equipe GPALT (Analistas de Cardápios)

Ativando uma conexão de banco de dados em memória usando o SGBD H2:

In [1]:
%defaultDatasource jdbc:h2:mem:db


# Importando dados processados

A partir dos notebooks anteriores, foi possível construir dados segundo o modelo proposto, integrando cada base da maneira que foi necessário. Resta, agora, construir os dados para o modelo final. O que falta será feito agora (por exemplo, receitas de bases diferentes precisam ser unidas em um único arquivo) seguindo a ordem mostrada no modelo relacional por simplicidade.

### Origem

Ainda é necessário associar, às origens das receitas, o tipo delas (se é uma região do mundo ou um país). A partir dos dados obtidos, isso foi verificado manualmente. Importando os dados considerando todos como regiões:

In [2]:
DROP TABLE IF EXISTS OrigemFlavor;
DROP TABLE IF EXISTS OrigemCulinary;

CREATE TABLE OrigemFlavor (
  Origem VARCHAR(50),
  Tipo VARCHAR(50),
  PRIMARY KEY (Origem)
) AS SELECT
  Origem, 'Regiao'
FROM CSVREAD('../data/interim/flavor network/origens.csv');

CREATE TABLE OrigemCulinary (
  Origem VARCHAR(50),
  Tipo VARCHAR(50),
  PRIMARY KEY (Origem)
  ) AS SELECT
  Origem, 'Regiao'
FROM CSVREAD('../data/interim/culinarydb/origens.csv');


Visualizando:

In [3]:
SELECT * FROM OrigemFlavor LIMIT 10;


In [4]:
SELECT * FROM OrigemCulinary LIMIT 10;


Atualizando o tipo dos países:

In [5]:
UPDATE OrigemFlavor
SET Tipo = 'Pais'
WHERE Origem = 'Misc.: Belgian' OR Origem = 'Misc.: Dutch' OR Origem = 'Misc.: Portugal';

UPDATE OrigemCulinary
SET Tipo = 'Pais'
WHERE Origem = 'Canada' OR Origem = 'China' OR Origem = 'France' OR Origem = 'Greece' OR Origem = 'Italy'
OR Origem = 'Japan' OR Origem = 'Mexico' OR Origem = 'Spain' OR Origem = 'Thailand' OR Origem = 'USA';


Unindo todas as origens em uma única VIEW:

In [6]:
DROP VIEW IF EXISTS OrigemFinal;

CREATE VIEW OrigemFinal AS
    SELECT * FROM OrigemFlavor
    UNION
    SELECT * FROM OrigemCulinary
    ORDER BY Origem;


Visualizando:

In [7]:
SELECT * FROM OrigemFinal LIMIT 10;


Exportando os dados

In [8]:
CALL CSVWRITE('../data/processed/database/origem.csv',
              'SELECT * FROM OrigemFinal');


37

### Receita

Nos notebooks anteriores, houve um trabalho de padronizar os dados das receitas segundo o modelo. Assim, resta importar esses dados e juntar em um único arquivo csv.

In [9]:
DROP TABLE IF EXISTS ReceitasFlavor;
DROP TABLE IF EXISTS ReceitasCulinary;
DROP TABLE IF EXISTS ReceitasRecipe;

CREATE TABLE ReceitasFlavor (
  IDReceita INT,
  NomeReceita VARCHAR(200),
  BancoOriginal VARCHAR(20),
  Origem VARCHAR(30),
  PRIMARY KEY (IDReceita)
) AS SELECT
  IDReceita, Nome, BancoOriginal, Origem
FROM CSVREAD('../data/interim/flavor network/receitas.csv');

CREATE TABLE ReceitasCulinary (
  IDReceita INT,
  NomeReceita VARCHAR(200),
  BancoOriginal VARCHAR(20),
  Origem VARCHAR(30),
  PRIMARY KEY (IDReceita)
) AS SELECT
  IDReceita, Nome, BancoOriginal, Origem
FROM CSVREAD('../data/interim/culinarydb/receitas.csv');

CREATE TABLE ReceitasRecipe (
  IDReceita INT,
  NomeReceita VARCHAR(200),
  BancoOriginal VARCHAR(20),
  Origem VARCHAR(30),
  PRIMARY KEY (IDReceita)
) AS SELECT
  ID, Nome, BancoOriginal, Origem
FROM CSVREAD('../data/interim/recipenlg/receitasfinais.csv');


Visualizando:

In [10]:
SELECT * FROM ReceitasFlavor LIMIT 10;


In [11]:
SELECT * FROM ReceitasCulinary LIMIT 10;


In [12]:
SELECT * FROM ReceitasRecipe LIMIT 10;


Unindo todas as receitas em uma única VIEW:

In [13]:
DROP VIEW IF EXISTS ReceitaFinal;

CREATE VIEW ReceitaFinal AS
    SELECT * FROM ReceitasFlavor
    UNION
    SELECT * FROM ReceitasCulinary
    UNION
    SELECT * FROM ReceitasRecipe
    ORDER BY IDReceita;


Visualizando:

In [14]:
SELECT * FROM ReceitaFinal LIMIT 10;


Exportando os dados:

In [15]:
CALL CSVWRITE('../data/processed/database/receita.csv',
              'SELECT * FROM ReceitaFinal');


255224

### Ingrediente

Os ingredientes finais da base são os construídos no notebook de análises do FooDB e os ingredientes compostos da base do CulinaryDB:

In [16]:
DROP TABLE IF EXISTS IngredientesSimples;
DROP TABLE IF EXISTS IngredientesCompostos;

CREATE TABLE IngredientesSimples (
  Nome VARCHAR(80),
  Classificacao VARCHAR(50),
  PRIMARY KEY (Nome)
) AS SELECT
  NOME, CLASSIFICACAO
FROM CSVREAD('../data/interim/foodb/ingredientes.csv');

CREATE TABLE IngredientesCompostos (
  Nome VARCHAR(80),
  Classificacao VARCHAR(50),
  PRIMARY KEY (Nome)
) AS SELECT
  Compound_Ingredient_Name, Category
FROM CSVREAD('../data/interim/culinarydb/dadoscompostos.csv');


Visualizando:

In [17]:
SELECT * FROM IngredientesSimples LIMIT 10;


In [18]:
SELECT * FROM IngredientesCompostos LIMIT 10;


Juntando os ingredientes em uma única VIEW:

In [19]:
DROP VIEW IF EXISTS Ingrediente;

CREATE VIEW Ingrediente AS
    SELECT * FROM IngredientesSimples
    UNION
    -- Somente realizar a união de ingredientes que já não estejam catalogados no FooDB
    (SELECT * FROM IngredientesCompostos WHERE Nome NOT IN (SELECT Nome FROM IngredientesSimples))
    ORDER BY Nome;


Visualizando:

In [20]:
SELECT * FROM Ingrediente LIMIT 10;


As classificações dos ingredientes compostos ainda não foram definidas em um arquivo próprio:

In [21]:
CALL CSVWRITE('../data/interim/culinarydb/classificacoescompostos.csv',
              'SELECT DISTINCT Classificacao FROM IngredientesCompostos');


12

Exportando os dados dos ingredientes:

In [22]:
CALL CSVWRITE('../data/processed/database/ingrediente.csv',
              'SELECT * FROM Ingrediente');


1081

### IngredienteComposto

Acontece de alguns ingredientes compostos do CulinaryDB já existirem na base do FooDB e, por isso, já terem quantidades de nutrientes associadas. Assim, esses ingredientes devem ser considerados como ingredientes simples no modelo. Os que não existirem ainda precisam ter seus ingredientes base associados.

Importando os dados:

In [23]:
DROP TABLE IF EXISTS AssociacaoCompostosSimples;

CREATE TABLE AssociacaoCompostosSimples (
  IngredienteComposto VARCHAR(80),
  IngredienteOriginal VARCHAR(80),
  PRIMARY KEY (IngredienteComposto, IngredienteOriginal)
) AS SELECT
  INGREDIENTECOMPOSTO, INGREDIENTEORIGINAL
FROM CSVREAD('../data/interim/culinarydb/ingredientecomposto.csv');


Visualizando:

In [24]:
SELECT * FROM AssociacaoCompostosSimples LIMIT 10;


Desconsiderando ingredientes que sejam ingredientes simples do FooDB:

In [25]:
DELETE FROM AssociacaoCompostosSimples
WHERE IngredienteComposto IN (SELECT Nome FROM IngredientesSimples);


Exportando esses dados:

In [26]:
CALL CSVWRITE('../data/processed/database/ingredientecomposto.csv',
              'SELECT * FROM AssociacaoCompostosSimples ORDER BY IngredienteComposto');


406

### Sabor

Os dados dos sabores serão montados a partir dos dados do Flavor Network. Um problema é que, mesmo no Flavor Network, alguns sabores são formados por ingredientes que não fazem parte da base. Considerando o modelo relacional proposto, não faz sentido ter sabores relacionados a um ingrediente que não está na base. Assim, os sabores só existirão se eles relacionarem dois ingredientes da base, ou seja, ingredientes cuja integração com o FooDB foi feita.

Importando os sabores e os ingredientes finais (agora construídos na base final)

In [27]:
DROP TABLE IF EXISTS Ingredientes;
DROP TABLE IF EXISTS Sabores;

CREATE TABLE Ingredientes (
  Nome VARCHAR(80),
  PRIMARY KEY (Nome)
) AS SELECT
  Nome
FROM CSVREAD('../data/processed/database/ingrediente.csv');

CREATE TABLE Sabores (
  ID INT,
  A VARCHAR(80),
  B VARCHAR(80),
  Quantidade INT,
  PRIMARY KEY (ID)
) AS SELECT
  ID, A, B, QUANTIDADE
FROM CSVREAD('../data/interim/flavor network/saboresfinais.csv');


Visualizando:

In [28]:
SELECT * FROM Ingredientes LIMIT 10;


In [29]:
SELECT * FROM Sabores LIMIT 10;


Apagando os sabores que contêm algum ingrediente que não faz parte da lista final:

In [30]:
DELETE FROM Sabores S
WHERE S.A NOT IN (SELECT Nome FROM Ingredientes);

DELETE FROM Sabores S
WHERE S.B NOT IN (SELECT Nome FROM Ingredientes);


Exportando

In [31]:
CALL CSVWRITE('../data/processed/database/sabor.csv',
              'SELECT * FROM Sabores');


41430

### Classificacao

A classificação dos ingredientes segundo a hierarquia já foi feita no notebook de análises do FooDB. No entanto, ainda é preciso considerar as classificações dos ingredientes compostos do CulinaryDB.

In [32]:
DROP TABLE IF EXISTS Classificacao;

CREATE TABLE Classificacao (
  Nome VARCHAR(100),
  CategoriaSuperior VARCHAR(100),
  PRIMARY KEY (Nome)
) AS SELECT
  CategoriaInferior, CategoriaSuperior
FROM CSVREAD('../data/interim/foodb/classificacao.csv');


Visualizando:

In [33]:
SELECT * FROM Classificacao LIMIT 10;


Inserindo as classificações do CulinaryDB:

In [34]:
INSERT INTO Classificacao(Nome)
SELECT DISTINCT Classificacao
FROM IngredientesCompostos;


Exportando:

In [35]:
CALL CSVWRITE('../data/processed/database/classificacao.csv',
              'SELECT * FROM Classificacao');


132